In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# 1️⃣ Generate correlated dataset
np.random.seed(42)
n_samples = 500
X = np.random.rand(n_samples, 7)
y = 5*X[:,0] + 3*X[:,1] + 2*X[:,2] + np.random.randn(n_samples)*0.3
for i in range(1, 7):
    X[:, i] = X[:, 0] + np.random.normal(0, 0.05, n_samples)

# Normalize features (important!)
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2️⃣ Ridge Regression (gradient descent)
def ridge_regression(X, y, lr, lam, epochs=2000):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0.0
    for epoch in range(epochs):
        y_pred = X.dot(weights) + bias
        dw = (-2/m) * X.T.dot(y - y_pred) + 2 * lam * weights
        db = (-2/m) * np.sum(y - y_pred)
        weights -= lr * dw
        bias -= lr * db
        
        # stop if weights explode
        if np.any(np.isnan(weights)) or np.any(np.isinf(weights)):
            return np.zeros(n), 0.0
    return weights, bias

def cost_function(X, y, weights, bias, lam):
    y_pred = X.dot(weights) + bias
    mse = np.mean((y - y_pred)**2)
    reg = lam * np.sum(weights**2)
    return mse + reg

# 3️⃣ Tune parameters
lrs = [0.0001, 0.001, 0.01, 0.1]
lambdas = [1e-10, 1e-5, 1e-3, 0.1, 1, 10]

best_r2, best_params = -999, None

for lr in lrs:
    for lam in lambdas:
        w, b = ridge_regression(X_train, y_train, lr, lam)
        y_pred = X_test.dot(w) + b
        if np.any(np.isnan(y_pred)):
            continue
        cost = cost_function(X_train, y_train, w, b, lam)
        r2 = r2_score(y_test, y_pred)
        if r2 > best_r2:
            best_r2, best_params = r2, (lr, lam, cost)

print("Best learning rate:", best_params[0])
print("Best lambda:", best_params[1])
print("Min cost:", best_params[2])
print("Max R²:", best_r2)


Best learning rate: 0.1
Best lambda: 1e-10
Min cost: 1.1414133408104115
Max R²: 0.62307632731881


/var/folders/nw/lnw4spfn70d50s0d4wxwldxr0000gn/T/ipykernel_34182/1654933623.py:28: RuntimeWarning: overflow encountered in dot
  dw = (-2/m) * X.T.dot(y - y_pred) + 2 * lam * weights


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# ✅ Working dataset URL
url = "https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Hitters.csv"
data = pd.read_csv(url)

# (a) Handle nulls and categorical encoding
data = data.dropna()
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# (b) Separate input and output, and scale features
X = data.drop('Salary', axis=1)
y = data['Salary']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# (c) Fit models
lin = LinearRegression().fit(X_train, y_train)
ridge = Ridge(alpha=0.5748).fit(X_train, y_train)
lasso = Lasso(alpha=0.5748).fit(X_train, y_train)

# (d) Evaluate
models = {'Linear': lin, 'Ridge': ridge, 'Lasso': lasso}
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name} → R²: {r2_score(y_test, y_pred):.3f}, MSE: {mean_squared_error(y_test, y_pred):.2f}")


Linear → R²: 0.290, MSE: 128431.59
Ridge → R²: 0.298, MSE: 126891.79
Lasso → R²: 0.298, MSE: 126974.95


/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.248e+04, tolerance: 4.367e+03
  model = cd_fast.enet_coordinate_descent(


In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

# Load dataset (modern replacement for Boston)
data = fetch_california_housing()
X, y = data.data, data.target

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# RidgeCV and LassoCV
ridge_cv = RidgeCV(alphas=[0.01, 0.1, 1, 10, 100]).fit(X_train, y_train)
lasso_cv = LassoCV(alphas=[0.01, 0.1, 1, 10, 100], max_iter=10000, cv=5).fit(X_train, y_train)

# Evaluate models
for name, model in {"RidgeCV": ridge_cv, "LassoCV": lasso_cv}.items():
    y_pred = model.predict(X_test)
    print(f"{name} Best Alpha: {model.alpha_}")
    print(f"{name} R²: {r2_score(y_test, y_pred):.3f}")
    print(f"{name} MSE: {mean_squared_error(y_test, y_pred):.3f}\n")


RidgeCV Best Alpha: 1.0
RidgeCV R²: 0.576
RidgeCV MSE: 0.556

LassoCV Best Alpha: 0.01
LassoCV R²: 0.582
LassoCV MSE: 0.548



In [8]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# One-vs-Rest Logistic Regression (manual implementation)
def one_vs_rest(X, y, num_classes, lr=0.1, epochs=1000):
    m, n = X.shape
    W = np.zeros((num_classes, n))
    b = np.zeros(num_classes)
    
    for c in range(num_classes):
        y_c = np.where(y == c, 1, 0)
        for _ in range(epochs):
            z = X.dot(W[c]) + b[c]
            h = sigmoid(z)
            W[c] -= lr * (1/m) * X.T.dot(h - y_c)
            b[c] -= lr * (1/m) * np.sum(h - y_c)
    return W, b

# Predict function
def predict(X, W, b):
    z = X.dot(W.T) + b
    probs = sigmoid(z)
    return np.argmax(probs, axis=1)

# Train model
num_classes = len(np.unique(y))
W, b = one_vs_rest(X_train, y_train, num_classes)

# Test model
y_pred = predict(X_test, W, b)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9666666666666667
